<a href="https://colab.research.google.com/github/vitorkol/association-rules/blob/master/regras_de_associacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importanto as bibliotecas**

In [10]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [11]:
#Lendo o dataset a ser utilizado no exercício
url = "https://raw.githubusercontent.com/vitorkol/association-rules/master/dataset/mercado.csv"
base_mercado = pd.read_csv(url, sep = ';')

**Identificando os dados e seus repectivos tipos**

Essa análise inicial permite ter uma visão dos tipos de daos e os possíveis tipos de tratamento.

In [ ]:
base_mercado.info()

In [ ]:
base_mercado.describe()

In [ ]:
base_mercado

**Tratamento de dados**</br></br>
O algoritmo de regras de associação necessita que o input de dados seja em formato de lista contendo indicador de true or false para cada produto.

Desta forma, temos que realizar tratamento de dados no dataset para satisfazer essa premissa.

Diante dessa necessidade a solução implementada é a transposição de linhas para colunas.

In [16]:
# Visualizando tamanho do dataset
len(base_mercado)

10

In [17]:
#Acessando o produto dentro da lista
print(base_mercado.values[1,1]) #Exibindo a informação da Linha 0 Coluna 3

cafe


In [19]:
# Temos que acessar cada item dentro do dataframe e inserir em uma lista
lista_compras = []
for i in range(len(base_mercado)):  
    lista_compras.append([str(base_mercado.values[i, j]) for j in range(4)])

In [ ]:
#Exibindo a listagem de compras gerada
lista_compras

**Criando lista de produtos com indicador de true or false**


Nesta etapa vamos criar a lista com todos os produtos do dataset e informando a transação e seus respectivos itens.

In [21]:
te = TransactionEncoder()
te_array = te.fit(lista_compras).transform(lista_compras)

In [ ]:
#Exibindo a listagem de produtos criada no array com valores True and False
te_array

**Criando dataframe com itens do mercado**


Vamos criar o dataframe através da lista criada e infomando as colunas que representam os itens do dataset.

In [23]:
#Neste processo de Transformação definimos o cabeçalho do dataset com o nome dos itens
df_prod_mercado = pd.DataFrame(te_array, columns = te.columns_)

In [ ]:
#Visualizando o dataframe
df_prod_mercado

Pode-se Obserevar que ao transpor as linhas em colunas os dados, das colunas que possuiam 
valores ausentes do tipo "Nan" foram incluidads como um produto do mercado. No entanto, isso não é verdadeiro. Dessa forma, vamos eliminar essa coluna do nosso dataframe.

In [25]:
df_prod_mercado.drop(columns='nan', inplace=True)

In [ ]:
#Exibindo o conjunto de dados e validando a remoção da coluna Nan.
df_prod_mercado

In [30]:
#Renomeando colunas para melhor entendimento dos dados
df_prod_mercado.rename(columns={'arroz': 'Arroz', 'cafe': 'Cafe','cerveja':'Cerveja', 'feijao':'Feijao', 'leite':'Leite','manteiga':'Manteiga','pao':'Pao'}, inplace = True)

**Gerando os itens frequentes no dataset**


Nesta etapa vamos gerar os itens sets mais frequentes no nosso dataframe. Para isso, vamos utiizar a biblioteca apriori.

In [32]:
from mlxtend.frequent_patterns import apriori

In [34]:
#Selecionando itens aplicando a métrica com o suporte de 30% nesse experimento.
frequent_itemsets =  apriori(df_prod_mercado, min_support=0.3, use_colnames=True)

In [ ]:
frequent_itemsets

**Criando as regras de associação**


Após a criação dos itens mais frequentes, o proximo passo é criar as regras de associação. Vamos importar a biblioteca association_rules para gerar as regras.

In [37]:
from mlxtend.frequent_patterns import association_rules

In [38]:
#Definindo as regras de associação a serem aplicadas no modelo.
regras = association_rules(frequent_itemsets, metric='confidence',min_threshold=0.8) # 80% regras acontecerem
#visualizando as regras criadas
regras

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cafe),(Manteiga),0.3,0.5,0.3,1.0,2.0,0.15,inf
1,(Cafe),(Pao),0.3,0.5,0.3,1.0,2.0,0.15,inf
2,(Manteiga),(Pao),0.5,0.5,0.4,0.8,1.6,0.15,2.5
3,(Pao),(Manteiga),0.5,0.5,0.4,0.8,1.6,0.15,2.5
4,"(Cafe, Manteiga)",(Pao),0.3,0.5,0.3,1.0,2.0,0.15,inf
5,"(Cafe, Pao)",(Manteiga),0.3,0.5,0.3,1.0,2.0,0.15,inf
6,(Cafe),"(Manteiga, Pao)",0.3,0.4,0.3,1.0,2.5,0.18,inf


In [39]:
#Selecionando colunas a serem analisadas
regras = regras[['antecedents','consequents', 'support','confidence','lift']]

In [40]:
#Renomeando colunas para melhor interpretação dos dados
regras = regras.rename(
    columns={
        "antecedents": "se",
        "consequents": "então",
        "support": "Suporte", 
        "confidence": "Confiança",
        "lift": "LIFT"
    }
)

In [ ]:
#Visuzalizando as alterações
regras

In [43]:
#Filtrando as melhores regras baseado no LIFT
regras.sort_values(by = 'LIFT', ascending = False)

,se,então,Suporte,Confiança,LIFT
6,(Cafe),"(Manteiga, Pao)",0.3,1.0,2.5
0,(Cafe),(Manteiga),0.3,1.0,2.0
1,(Cafe),(Pao),0.3,1.0,2.0
4,"(Cafe, Manteiga)",(Pao),0.3,1.0,2.0
5,"(Cafe, Pao)",(Manteiga),0.3,1.0,2.0
2,(Manteiga),(Pao),0.4,0.8,1.6
3,(Pao),(Manteiga),0.4,0.8,1.6


**Conclusão da análise dos Resultados obtidos**


A tabela acima demonstra as regras de associação criadas através do dataset de produtos. Através dela, podemos observar os itens sets e seus relacionamentos. Bem como, suas metricas de suporte, confiança e LIFT.

**Exemplo de leitura da tabela:**

Analisando a primeira linha do dataframe: Se (café) então (manteiga e pão)

- Clientes que compram café tem 2,5 vezes mais chances de comprarem manteiga e pão;
- Em 100% dos casos, clientes que compram café também compram manteiga e pão;